In [2]:
import random                        # Library to generate random numbers
import skimage                       # Library for image manipulation
import numpy as np                   # Library for array manipulation
import urllib.request                # Library to download data
import matplotlib.pyplot as plt      # Library used for plotting
from skimage import io, measure               # Module from skimage
from skimage.io import imread        # Module from skimage to read images as numpy arrays
from skimage.filters import gaussian # Module working with a gaussian filter
import pathlib                              # Library to work with file paths
import os
from ipywidgets import interactive, HBox, VBox, Layout
import ipywidgets as widgets
from skimage.morphology import binary_dilation
from skimage.segmentation import watershed
from skimage.draw import polygon
from skimage.measure import regionprops
from skimage.color import label2rgb
from skimage.filters import threshold_otsu
from skimage.morphology import binary_erosion
from skimage.morphology import binary_closing
from skimage.morphology import binary_opening
from skimage.morphology import disk
from skimage.morphology import remove_small_objects
from scipy import ndimage as ndi              # Distance Transform
from skimage.feature import peak_local_max    # Local maxima in a matrix
from skimage.segmentation import watershed    # Watershed algorithm
from skimage.filters import difference_of_gaussians
from cellpose import plot, models
import pandas as pd

In [15]:
tif_dir = 'C:/Users/apurv/OneDrive/Desktop/QBio/Challenges/UQbio2024_Team5/NoDrug/'
file_dict  = {
    'FOV1':'rep1_FOV1.tif',
    'FOV2':'rep1_FOV2.tif',
    'FOV3':'rep1_FOV3.tif',
    'FOV4':'rep1_FOV4.tif',
    'FOV5':'rep1_FOV5.tif',
    'FOV6':'rep1_FOV6.tif'
    # Add more file paths as needed
}

In [18]:
answerDict = {
    'FOV': [],
    'CellCount': [],
    'CellArea': [],
    'mRNA_nucleus': [],
    'mRNA_cytosol': []
}

In [19]:
# Loop through each file
for var_name, file_name in file_dict.items():
    file_path = os.path.join(tif_dir, file_name)
    
    # Load the image
    images = imread(file_path)
    img = images[0, :, :, 0]

    # RUN CELLPOSE
    model = models.Cellpose(model_type='cyto')  # model_type='cyto' or model_type='nuclei'
    masks = model.eval(img, diameter=200)[0]
    num_cells = len(np.unique(masks)) - 1

    # Segment nuclei and cytosol
    zSlice = 10
    images_FISH = images
    img_nuc = images_FISH[zSlice, :, :, 0:2]
    use_GPU = False  # Set to True if you have a GPU - this will make it MUCH faster.
    model_nuc = models.Cellpose(gpu=use_GPU, model_type='nuclei')  # model_type='cyto' or model_type='nuclei'
    masks_nuc = model_nuc.eval(img_nuc, diameter=100, channels=[0, 1])[0]

    img_cyto = images_FISH[zSlice, :, :, 0:3]
    model_cyto = models.Cellpose(gpu=use_GPU, model_type='cyto2')  # model_type='cyto', 'cyto2' or model_type='nuclei'
    masks_cyto, flows, styles, diams = model_cyto.eval(img_cyto, diameter=200, channels=[0, 2])

    nucleus_indices = np.zeros(np.max(masks_cyto) + 1)
    for i in range(1, np.max(masks_nuc) + 1):
        posn_nucl = np.mean(np.where(masks_nuc == i), axis=1).astype(int)
        if posn_nucl[0] < masks_cyto.shape[0] and posn_nucl[1] < masks_cyto.shape[1]:
            nucleus_indices[masks_cyto[posn_nucl[0], posn_nucl[1]]] = i

    # Counting the detected particles in each cell
    list_cytosol_particles = np.zeros(np.max(masks_cyto))
    list_nuclear_particles = np.zeros(np.max(masks_cyto))

    # Selecting the color channel with RNA spots
    zSlice = 10  # time frame
    iChannel = 1  # which channel
    img_spots = images_FISH[zSlice, :, :, iChannel]

    # Apply a difference of Gaussians filter to the image to enhance spots
    img_spots_filtered = difference_of_gaussians(img_spots, low_sigma=1, high_sigma=5)

    threshold = 0.05
    img_spots_binary = img_spots_filtered.copy()
    img_spots_binary[img_spots_binary >= threshold] = threshold  # Making spots above the threshold equal to the threshold value.
    img_spots_binary[img_spots_binary < threshold] = 0  # Making spots below the threshold equal to 0.
    img_spots_binary[img_spots_binary != 0] = 1  # Binarization

    # Labeling. Joining pixels in "particles"
    spot_contours = measure.find_contours(img_spots_binary, 0.5)

    # Loop through the cytosol masks
    for i in range(len(spot_contours)):
        # Find the position of the particle
        posn = np.mean(spot_contours[i], axis=0).astype(int)
        if posn[0] < masks_cyto.shape[0] and posn[1] < masks_cyto.shape[1]:  # Ensure within bounds
            cell_num = masks_cyto[posn[0], posn[1]]
            if cell_num > 0 and cell_num <= num_cells:
                list_cytosol_particles[cell_num - 1] += 1

                # Check if the particle is also in the nucleus
                if masks_nuc[posn[0], posn[1]] == nucleus_indices[cell_num]:
                    list_nuclear_particles[cell_num - 1] += 1

    # Fill the answer dictionary
    for cell_id in range(1, num_cells + 1):
        cell_area = np.sum(masks_cyto == cell_id)
        mRNA_nucleus = list_nuclear_particles[cell_id - 1]
        mRNA_cytosol = list_cytosol_particles[cell_id - 1] - mRNA_nucleus
        
        answerDict['FOV'].append(var_name)
        answerDict['CellCount'].append(cell_id)
        answerDict['CellArea'].append(cell_area)
        answerDict['mRNA_nucleus'].append(mRNA_nucleus)
        answerDict['mRNA_cytosol'].append(mRNA_cytosol)


In [20]:
# Convert the dictionary to a DataFrame
df = pd.DataFrame(answerDict)

# Display the DataFrame
print(df)

     FOV  CellCount  CellArea  mRNA_nucleus  mRNA_cytosol
0   FOV1          1     11671           0.0           0.0
1   FOV1          2      9176           2.0           0.0
2   FOV1          3     10349           2.0           0.0
3   FOV1          4     10717           0.0           3.0
4   FOV1          5     10045           0.0          11.0
..   ...        ...       ...           ...           ...
60  FOV6          8     10838           0.0           5.0
61  FOV6          9      9022           0.0          12.0
62  FOV6         10      9250           0.0           0.0
63  FOV6         11      9569           2.0           0.0
64  FOV6         12     10047           0.0           0.0

[65 rows x 5 columns]


In [22]:
print(df.to_markdown())

|    | FOV   |   CellCount |   CellArea |   mRNA_nucleus |   mRNA_cytosol |
|---:|:------|------------:|-----------:|---------------:|---------------:|
|  0 | FOV1  |           1 |      11671 |              0 |              0 |
|  1 | FOV1  |           2 |       9176 |              2 |              0 |
|  2 | FOV1  |           3 |      10349 |              2 |              0 |
|  3 | FOV1  |           4 |      10717 |              0 |              3 |
|  4 | FOV1  |           5 |      10045 |              0 |             11 |
|  5 | FOV1  |           6 |      10822 |              0 |             14 |
|  6 | FOV1  |           7 |      10477 |              0 |              0 |
|  7 | FOV1  |           8 |      10185 |              2 |              0 |
|  8 | FOV1  |           9 |       8228 |              2 |              0 |
|  9 | FOV1  |          10 |      10080 |              2 |              0 |
| 10 | FOV1  |          11 |       9035 |              0 |             13 |
| 11 | FOV1 